# I, Import librabries



In [ ]:
from langchain_openai import AzureChatOpenAI
import requests
from crawl4ai import AsyncWebCrawler
from typing import List, Dict
from pymilvus.model.sparse.bm25.tokenizers import build_default_analyzer
from pymilvus.model.sparse import BM25EmbeddingFunction
import scipy.sparse as sp
from output import example_doc,duplicate
api_key = 'GOOGLE SEARCH API KEY'
search_engine_id = 'GOOGLE SEARCH ENGINE ID'
from pymilvus import DataType, MilvusClient
from pymilvus import (
    MilvusClient,
    DataType,
)
import feedparser
from datetime import datetime, timedelta
import time
from bs4 import BeautifulSoup
from datetime import timezone


In [35]:
# Initialize session
session = requests.Session()

# II, Vector Databse

## 1, Create Collection and Schema

In [ ]:
uri = "/Users/qa.nguyen/Documents/Intern/Viettel/WebScrapper/MyWebThinker/myvilvus.db"
collection_name = "mycollection"
client = MilvusClient(uri=uri)
from pymilvus.model.sparse.bm25.tokenizers import build_default_analyzer
from pymilvus.model.sparse import BM25EmbeddingFunction
analyzer = build_default_analyzer(language="en")
bm25_ef = BM25EmbeddingFunction(analyzer)
analyzer_params = {"tokenizer": "standard", "filter": ["lowercase"]}
schema = MilvusClient.create_schema()
schema.add_field(
    field_name="id",
    datatype=DataType.VARCHAR,
    is_primary=True,
    auto_id=True,
    max_length=100,
)
schema.add_field(
    field_name="content",
    datatype=DataType.VARCHAR,
    max_length=65535,
    analyzer_params=analyzer_params,
    enable_match=True,  # Enable text matching
    enable_analyzer=True,  # Enable text analysis
)
schema.add_field(
    field_name="link",
    datatype=DataType.VARCHAR,
    max_length=100
)
schema.add_field(field_name="sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR)
index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    field_name="sparse_vector",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="IP",
)
if client.has_collection(collection_name):
    client.drop_collection(collection_name)
# Create the collection
client.create_collection(
    collection_name=collection_name,
    schema=schema,
    index_params=index_params,
)

print(f"Collection '{collection_name}' created successfully")


/opt/anaconda3/envs/mental_health/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Collection 'mycollection' created successfully


## 2, Create Vector Embeddings

In [ ]:
def vectorEmbedding(collection_name, docs, links):
    """
    Input:
    - collection_name: collection name for storing vector db
    - docs: list of documents to be embedded
    - links: list of links need to be stored into db
    Output: no output, the function just insert data into database

    """
    sparse_dicts = []
    bm25_ef.fit(docs)
    docs_embeddings = bm25_ef.encode_documents(docs)
    for vec in docs_embeddings:
        if sp.issparse(vec):
            coo = vec.tocoo()
            sparse_dicts.append({int(c): float(d) for c, d in zip(coo.col, coo.data)})
        else:  # assume dense numpy array
            sparse_dicts.append({i: float(v) for i, v in enumerate(vec) if v != 0.0})
    entities = []

    for i in range (len(docs)):
        entities.append(
            {
                "content": docs[i],
                "link": links[i],
                "sparse_vector": sparse_dicts[i],   # <-- include your BM25 vector here
            }
        )

    # Insert data
    client.insert(collection_name, entities)

## 3, Remove Duplicate Links

In [ ]:

def remove_duplicates(links):
    """
    Input: A list of links
    Output: A list of unique links
    """
    if not links:
        return []
    else:
        for link in links:
            res = client.query(
    collection_name="mycollection",
    filter=f"link like \"{link}\"",
    output_fields=["link"],
    limit=1
            )
            if res:
                links.remove(link)
    return links

## 4, Check Repost Contents

In [ ]:
def repost(doc):
    return False

# III, Get Daily Papers From Arxiv

In [36]:

def get_arxiv_by_subject_and_dates(
    subject: str,
    start: datetime,
    end: datetime,
    max_results: int = 50
) -> List[Dict[str, str]]:
    """
    Query arXiv for papers in `subject` whose submission times fall
    between `start` and `end` (both datetimes in UTC).

    Returns a list of {"link": ..., "abstract": ...} dicts.
    """
    # Format datetimes as YYYYMMDDHHMM (GMT)
    fmt = "%Y%m%d%H%M"
    start_s = start.strftime(fmt)
    end_s   = end.strftime(fmt)

    # Build the search_query string
    # Use '+' for spaces/AND between terms in the URL
    date_filter = f"submittedDate:[{start_s}+TO+{end_s}]"
    query = f"cat:{subject}+AND+{date_filter}"

    # Construct the full API URL
    base_url = "http://export.arxiv.org/api/query?"
    url = (
        f"{base_url}"
        f"search_query={query}"
        f"&start=0"
        f"&max_results={max_results}"
        f"&sortBy=submittedDate"
        f"&sortOrder=descending"
    )

    # Fetch and parse the Atom feed
    feed = feedparser.parse(url)
    links = []
    abstracts = []
    for entry in feed.entries:
        links.append(entry.id)
        abstracts.append(entry.summary.strip().replace("\n", " "))
    return links, abstracts


# IV, Get Daily Trending Repo From Github
- This can change to be weekly

In [ ]:

TRENDING_URL = "https://github.com/trending?since=daily"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    )
}

def fetch_trending_repos(max_repos: int = 20) -> List[str]:
    """Return a list like ["owner1/repo1", "owner2/repo2", …]."""
    resp = requests.get(TRENDING_URL, headers=HEADERS, timeout=15)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    links = soup.select("article.Box-row h2 a")  # repo links
    repos = []
    for a in links[:max_repos]:
        # href looks like "/owner/repo"
        repo_path = a["href"].strip("/")
        repos.append(repo_path)
    return repos


def grab_readme(owner_repo: str) -> Dict[str, str]:
    """
    Attempt to fetch the raw README from the repo’s default branch (main/master).
    Returns dict {"repo": "<owner/repo>", "url": "<raw-url>", "content": "<md or ''>"}
    """
    owner, repo = owner_repo.split("/")
    branches = ["main", "master"]
    content = ""
    raw_url = ""
    for br in branches:
        url = f"https://raw.githubusercontent.com/{owner}/{repo}/{br}/README.md"
        r = requests.get(url, headers=HEADERS, timeout=15)
        if r.status_code == 200 and r.text.strip():
            raw_url = url
            content = r.text
            break
    return {"repo": owner_repo, "url": raw_url, "content": content}

def getTrendingRepo():
    top_repos = fetch_trending_repos(max_repos=10)
    readmeLinks = []
    readmeContent = []
    for idx, repo in enumerate(top_repos, 1):
        data = grab_readme(repo)
         
        if data["content"]:
            readmeLinks.append(data['url'])
            readmeContent.append(data["content"])
        else:
            print(f"[{idx}/{len(top_repos)}] {repo}: README not found on main/master.\n")
        time.sleep(1)   # polite delay
    return readmeLinks, readmeContent

# V, Useful Tools

- web_extract_sync: Extract Website content in markdown
- EnrichContent: Find related links and contents to the input post
- google_search: Function to search Google by query

In [5]:
#google search
def google_search(query, api_key=api_key,search_engine_id=search_engine_id, **params, ):
    """
    Perform an asynchronous search using the Google Search Engine.

    Args:
        query (str): Search query.
        api_search_key (str): key for the Google Search API.
        search_engine_id (str): key for Google Search Engine
    Returns:
        dict: JSON response of the search results. Returns empty dict if all retries fail.
    """
    query = query
    base_url = 'https://www.googleapis.com/customsearch/v1'
    params = {
        'key': api_key,
        'cx': search_engine_id,
        'q': query,
        **params
    }
    response = session.get(base_url, params=params)
    response.raise_for_status
    search_result = response.json()
    return search_result

In [4]:
async def _web_extract_tool(urls: List[str]) -> List[dict]:
    """
    Async content‐extraction using crawl4ai. 
    Returns a list of dicts: {'link': <url>, 'raw_content': <markdown>}.
    """
    useful_info = []
    # We can reuse one single AsyncWebCrawler per URL to keep things simple.
    async with AsyncWebCrawler() as crawler:
            results = await crawler.arun_many(urls)
            for result in results:
                if result.success:
                    # Print to console so you see progress
                    useful_info.append(result.markdown)
                else:
                    # If crawl failed, still record the link with an error message
                    useful_info.append({
                        "link": result.url,
                        "raw_content": f"[FAILED to crawl]"
                    })
    return useful_info


In [6]:
import asyncio
def web_extract_sync(urls: List[str]) -> List[dict]:
    """
    Synchronous wrapper for _web_extract_tool.
    Internally runs the async function on an event loop and returns List[dict].
    """
    return asyncio.run(_web_extract_tool(urls))

In [ ]:

llm = AzureChatOpenAI(
    azure_endpoint="",
    api_key="",
    api_version="",
    deployment_name="gpt-4o",
    temperature=0.2,   # lower temperature to minimize hallucinations
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """
You’re given a piece of text describing a project, paper, or tool.  
Your job is to **extract the one most important keyword or name** that a user could plug into Google Scholar to find broad, relevant information.  

– Only output a single word or short phrase (1–5 words).  
– Do not include quotes, punctuation, or any explanation.  
– Make it as general as possible (e.g. “WebThinker” rather than “WebThinker deep research framework…”).

### Examples

Text:
“WebThinker: Empowering Large Reasoning Models with Deep Research Capability. … The code is available at https://github.com/RUC-NLPIR/WebThinker.”

Output:
WebThinker

Text:
“In recent years particle filters have been used for robot localization… we implement a hybrid landmark‐and‐wall‐based Bayesian filter in Webots.”

Output:
Particle filter localization

---

Text:
{post}

Output:
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", template),
    ("user", "{post}")
])


In [ ]:
def EnrichContent(docs):
    for doc in docs:
        prompt = prompt_template.invoke({"post": doc})
        response = llm.invoke(prompt)
        print(response.content)
        search_results = google_search(query=query)
        links = []
        if 'items' in search_results:
            for idx, item in enumerate(search_results['items'][:3]):
                link =  item.get('link', '')
                links.append(link)
        links = remove_duplicates(links)
        #links = repost(links)
        docs = web_extract_sync(links)
        vectorEmbedding("mycollection", docs, links)
    print(links)

In [38]:

# Define your two-day window (UTC)
today = datetime.now(timezone.utc)
two_days_ago = today.replace(hour=0, minute=0, second=0, microsecond=0) \
                    - timedelta(days=2)

ArxivLinks, ArxivAbstracts = get_arxiv_by_subject_and_dates(
    subject="cs.AI",
    start=two_days_ago,
    end=today,
    max_results=10 #20
)

GitHubLinks, GithubRePo = getTrendingRepo()





[6/10] eythaann/Seelen-UI: README not found on main/master.



In [ ]:
print(ArxivLinks)
print()

['http://arxiv.org/abs/2506.09050v1', 'http://arxiv.org/abs/2506.09049v1', 'http://arxiv.org/abs/2506.09046v1', 'http://arxiv.org/abs/2506.09040v1', 'http://arxiv.org/abs/2506.09038v1', 'http://arxiv.org/abs/2506.09034v1', 'http://arxiv.org/abs/2506.09033v1', 'http://arxiv.org/abs/2506.09027v1', 'http://arxiv.org/abs/2506.09018v1', 'http://arxiv.org/abs/2506.08999v1']
